In [2]:
# Uso del programa
binary_file = './StupidCrackMe.exe'  # Reemplaza con la ruta correcta de tu archivo binario
output_file = './disassembled.txt'  # Reemplaza con la ruta donde quieres guardar el resultado

In [5]:
import pefile
from capstone import *

def disassemble_binary(file_path):
    # Cargar el archivo PE
    pe = pefile.PE(file_path)
    
    # Inicializar el desensamblador de Capstone para arquitectura x86 (32 bits)
    md = Cs(CS_ARCH_X86, CS_MODE_32)
    
    disassembled_code = []
    
    # Buscar secciones ejecutables
    for section in pe.sections:
        # if section.Characteristics & 0x20000020:  # Verificar si la sección es ejecutable
        print(f"Sección {section.Name.decode().rstrip()} encontrada en el offset 0x{section.PointerToRawData:X}")
        
        # Obtener datos binarios de la sección
        code = section.get_data()
        
        # Desensamblar el código
        for instruction in md.disasm(code, section.VirtualAddress):
            disassembled_code.append(f"0x{instruction.address:X}:\t{instruction.mnemonic}\t{instruction.op_str}")
    
    return disassembled_code

def write_disassembled_code(disassembled_code, output_file):
    with open(output_file, 'w') as f:
        for line in disassembled_code:
            f.write(line + "\n")


disassembled_code = disassemble_binary(binary_file)
write_disassembled_code(disassembled_code, output_file)

print(f"El código desensamblado se ha guardado en {output_file}")

Sección .text    encontrada en el offset 0x400
Sección .rdata   encontrada en el offset 0x17200
Sección .data    encontrada en el offset 0x1B600
Sección .idata   encontrada en el offset 0x1CA00
Sección .reloc   encontrada en el offset 0x1D600
El código desensamblado se ha guardado en ./disassembled.txt


In [4]:
import r2pipe
import json

def disassemble_binary(file_path):
    r2 = r2pipe.open(file_path)
    r2.cmd('aaa')  # Analiza todo el binario
    functions = r2.cmdj('aflj')  # Obtiene la lista de funciones en formato JSON

    disassembled_code = []
    for func in functions:
        func_name = func['name']
        func_start = func['offset']
        disassembled_code.append(f"\nFunción {func_name} en 0x{func_start:X}:")

        # Desensamblar instrucciones en la función
        instructions = r2.cmdj(f'pdj 100 @ {func_start}')
        for instr in instructions:
            print(instr.keys())
            address = instr['offset']
            mnemonic = instr['mnemonic']
            op_str = instr['opcode']
            disassembled_code.append(f"0x{address:X}: {mnemonic} {op_str}")

    r2.quit()
    return disassembled_code

def write_disassembled_code(disassembled_code, output_file):
    with open(output_file, 'w') as f:
        for line in disassembled_code:
            f.write(line + "\n")

disassembled_code = disassemble_binary(binary_file)
write_disassembled_code(disassembled_code, output_file)

print(f"El código desensamblado y anotado se ha guardado en {output_file}")

dict_keys(['offset', 'esil', 'refptr', 'fcn_addr', 'fcn_last', 'size', 'opcode', 'disasm', 'bytes', 'family', 'type', 'reloc', 'type_num', 'type2_num', 'jump', 'fail', 'flags', 'refs'])


KeyError: 'mnemonic'